<a href="https://colab.research.google.com/github/GuilhermePelegrina/Mackenzie/blob/main/Aulas/2s2024/TIC/Aula_05_Aspectos_adicionais_train_test_lab_resolvido2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/logo_mackenzie.png'>


# **Atividade Prática - Aspectos adicionais sobre treinamento e teste**

# Base de dados: Heart Disease!


A objetivo da base de dados "Heart Disease" e de treinar um classificador caaz de detectar a presença de uma doença cardiovascular tendo um conjunto de informações acerca de pessoas que possuem ou não tal doença.

O conjunto de dados se encontra [neste link](https://raw.githubusercontent.com/GuilhermePelegrina/Mackenzie/main/Datasets/data_heart_statlog_cleveland.csv). A coluna *target* indica a classe a ser prevista, sendo o valor 1 a presença da doença e o valor 0 a ausência.

**Imortante** Reparem que, embora todas as informações dessa base de dados são numéricas, a coluna *sex* indica uma variável categórica. No caso, 1 indica homens e 0 mulheres. O atributo *chest pain type* também é categórico, mas, para esta atividade, considere-o como numérico.

Responda as questões no Moodle (Atividades Práticas - Aspectos adicionais Train/Test). Lembre-se de remover linhas com células vazias, se houver. Também não se esqueça de importar os pacotes que serão utilizados nessa atividade!

In [ ]:
spl = 0.25
randstat = 1

**Questão 1.** Considere todas as variáveis como variáveis de entrada (com exceção da *target*). Separe os dados em treinamento (75% dos dados) e teste (25% dos dados) usando o comando `train_test_split` e com `random_state = 1`. Estratifique em relação aos dados de saída. Construa um modelo de regressão logística para prever se um futuro paciente possui ou não uma doença cardiovascular. Para os dados de teste, qual foi a acurácia obtida? Use o Classification Report para responder essa questão.

In [ ]:
import pandas as pd
dados = pd.read_csv('https://raw.githubusercontent.com/GuilhermePelegrina/Mackenzie/main/Datasets/data_heart_statlog_cleveland.csv')
dados.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, ConfusionMatrixDisplay

# Separando variáveis independentes das dependentes
x = dados.drop('target', axis=1)
y = dados['target']

# Separando em treino e Teste (30% para Teste)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=spl, random_state=randstat, stratify=y)

# Declarando o modelo
model = LogisticRegression()

# Treinando o modelo (com o conjunto de Treino)
model.fit(pd.DataFrame(X_train),y_train)

# Utilizando o modelo para prever novos casos
y_pred=model.predict(pd.DataFrame(X_test))

report = classification_report(y_test, y_pred)
print(report)

**Questão 2.** Usando o resultado do modelo anterior, dentre as pessoas que possuiam a doença (nos dados de teste), qual o percentual das pessoas que foram erroneamente classificadas como quem não possui tal doença.

Resposta: 0.15

**Questão 3.** Ainda no modelo da questão 1, indique a Precisão do modelo em detectar a doença.

Resposta: 0.86

**Questão 4.** Ajuste, agora, um modelo de k vizinhos mais próximos, com k = 5, para detectar a doença cardiovascular (usando novamente todas as variáveis de entrada). Use 75% dos dados para treinar o modelo, estratifique na variável de saída e considere random_state = 1 no `train_test_split`. Para a normalização dos dados, divida pelo máximo. Obs: não precisa transformar variáveis em dummies nessa base de dados. Qual a acurácia obtida nos dados de teste? Use o Classification Report para responder essa questão.

In [ ]:
from sklearn import neighbors

# Dados de entrada/saída
X = dados.drop('target',axis=1)
y = dados['target']

# Separando os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=spl, random_state=randstat, stratify=y)

# padroniza os dados para que a média seja 0 e o desvio padrão seja 1.
X_train_padronizados = X_train/X_train.max()

# Definindo ou declarando o modelo
k=5
clf = neighbors.KNeighborsClassifier(k)

# Aprendizado (Emprega o conjunto de treinamento)
clf.fit(X_train_padronizados, y_train)

# Fazendo a predição
X_test_padronizados = X_test/X_train.max()
y_pred = clf.predict(X_test_padronizados)

report = classification_report(y_test, y_pred)
print(report)

**Questão 5.** Usando o resultado do modelo anterior, dentre as pessoas que foram classificadas como quem não possui a doença, qual o percentual das que foram erroneamente classificadas como quem possui tal doença.

Resposta: 0.14

**Questão 6.** Ainda no modelo da questão 4, indique a Sensibilidade (ou Recall) do modelo em relação aos que possuem a doença.

Resposta: 0.88

**Questão 7.** Considere, novamente, o modelo da questão 4. Calcule a diferença absoluta entre as acurácias obtidas em relação ao sexo dos indivíduos. Qual o valor obtido?

In [ ]:
accuracy = accuracy_score(y_test, y_pred)

accuracy_homens = accuracy_score(y_test[X_test['sex'] == 1], y_pred[X_test['sex'] == 1])

accuracy_mulheres = accuracy_score(y_test[X_test['sex'] == 0], y_pred[X_test['sex'] == 0])
print("Diferença absoluta:", abs(accuracy_homens-accuracy_mulheres))

**Questão 8.** Considere, novamente, como classificador o modelo de k vizinhos mais próximos. Use 75% dos dados para treinar o modelo, estratifique na variável de saída e considere random_state = 1 no `train_test_split`. Para a normalização dos dados, divida pelo máximo. Faça um programa que construa tal modelo para valores de k iguais a 3, 5, 7 e 9. Qual a melhor acurácia obtida nos dados de teste? Calcule a acurácia com base na matriz de confusão.

In [ ]:
from sklearn.model_selection import KFold # Importando a biblioteca do k-fold

k = [3,5,7,9] # Definindo o número de pastas

acuracia = [] # Lista vazia para guardar as acurácias

acuracia_atual = 0
melhor_acuracia = 0

for ii in k:
  # Dados de entrada/saída
  X = dados.drop('target',axis=1)
  y = dados['target']

  # Separando os dados
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=spl, random_state=randstat, stratify=y)

  # padroniza os dados para que a média seja 0 e o desvio padrão seja 1.
  X_train_padronizados = X_train/X_train.max()

  # Definindo ou declarando o modelo
  clf = neighbors.KNeighborsClassifier(ii)

  # Aprendizado (Emprega o conjunto de treinamento)
  clf.fit(X_train_padronizados, y_train)

  # Fazendo a predição
  X_test_padronizados = X_test/X_train.max()
  y_pred = clf.predict(X_test_padronizados)

  cnf_matrix = confusion_matrix(y_test, y_pred) # Matriz de confusão
  acuracia_atual = (cnf_matrix[0][0] + cnf_matrix[1][1])/len(y_pred) # Acurácia do modelo atual

  if len(acuracia) == 0:
    melhor_acuracia = acuracia_atual
  else:
    if acuracia_atual > melhor_acuracia:
      melhor_acuracia = acuracia_atual

  acuracia.append(acuracia_atual) # Atualiza a lista de acurácias

print(acuracia)
print(melhor_acuracia)